# Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
from sklearn.linear_model    import LinearRegression
from sklearn.metrics         import r2_score
from sklearn.metrics         import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline        import Pipeline
from IPython.core.display    import display, HTML
sns.set(style = "white", palette = "husl")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

# Table Of Contents

-----

1. [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)

-----

2. [Feature Engineering](#Feature-Engineering)
    - [Transforming Numeric Data](#Transforming-Numeric-Data)
    - [Creating Segmental Features](#Creating-Areal-Features)
    - [Implementing The Pattern Sub-Model Approach](#Implementing-The-Pattern-Sub-Model-Approach)

# Reading In The Data

In [ ]:
mri = pd.read_csv("../Data/mri_cleaned.csv")

## Overview

In [ ]:
mri.head()

The `smoker_status` column is still in the data, but we will not needing the column for the models because we turned it into a pair of dummy columns so we will drop the column here.

In [ ]:
#  Dropping `smoker_status`

mri = mri.drop("smoker_status", axis = 1)

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {mri.shape}")

In [ ]:
# Summary of column data types

mri.dtypes.value_counts()

In [ ]:
# Checking for columns with missing/NaN data

(mri.isnull().mean()*100).sort_values(ascending = False).head(4)

# Feature Engineering

## Transforming Numeric Data

There are only four numeric columns in the data set: `age`, `lvesv`, `lvedv`, `lvef`.  Of the four, only `lvef` does not have any kind of a normal distribution: `age` is close to normally distributed, while `lvesv` and `lvedv` are log-normally distributed.

We cannot do anything to `lvedv` because that is my target variable, but we can take the log of `lvesv` (in this case the natural log).  We also tried squaring `age` but that did not affect the distribution in the way we hoped it would.

In [ ]:
# Take the natural log of `lvesv`.
# We chose to make it it's own column rather
# than overwrite the column.

mri["lvesv_log"] = mri["lvesv"].apply(lambda x: np.log(x))

In [ ]:
print(f"The shape of the dataset is: {mri.shape}")

[Top](#Table-Of-Contents)

## Creating Segmental Features

Because the model attempts to predict the end diastolic volume, we want it to be as accurate as it can be.  As part of that, we will try to use many combinations of features in an attempt to achieve high accuracy.

The data have 34 columns that we wish to engineer: a column measuring scarification and a column measuring ischemia.  Because there are so many of them, we felt the need to experiment with how they are passed into the model.  We are unable to create interaction columns, because there are zeros.  Instead, we elected to create segmental columns by summing similar columns together: we will compare the model's performance with the originals and with the segmental columns.

We used this image to guide our create of segmental columns:

<img src = "../Images/cardiac-segmentation-for-cardiac-perfusion-defects.jpg" alt = "Cardiac Segmentation" height = 700 width = 700>

Case courtesy of Dr Hamid Chalian, <a href="https://radiopaedia.org/">Radiopaedia.org</a>. From the case <a href="https://radiopaedia.org/cases/47102">rID: 47102</a>

In [ ]:
# Creating columns based off of the red, blue, green, and yellow
# I'm combining the apex and apical regions because there is only
# one region in the apex.

mri["basal"]  = mri["ba_he"] + mri["bas_he"] + mri["bis_he"] \
                + mri["bi_he"] + mri["bil_he"] + mri["bal_he"]
mri["mid"]    = mri["ma_he"] + mri["mas_he"] + mri["mis_he"] \
                + mri["mi_he"] + mri["mil_he"] + mri["mal_he"]
mri["apical"] = mri["aa_he"] + mri["as_he"] + mri["ai_he"] \
                + mri["al_he"] + mri["apex_he"]

print(f"The shape of the dataset is: {mri.shape}")